# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of plugins that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.2.0"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.2.0-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.2.0 Microsoft.SemanticKernel.Planners.Handlebars, 1.2.0-preview

### Setting Up Handlebars Planner
Handlebars Planner is located in the `Microsoft.SemanticKernel.Planning.Handlebars` package.

In [2]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060
var planner = new HandlebarsPlanner(new HandlebarsPlannerOptions() { AllowLoops = true });

### Providing plugins to the planner
The planner needs to know what plugins are available to it. Here we'll import the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

In [3]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "plugins");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "WriterPlugin"));

#!import plugins/StaticTextPlugin.cs
kernel.ImportPluginFromType<StaticTextPlugin>();

Define your ASK. What do you want the Kernel to do?

In [4]:
#pragma warning disable SKEXP0060

var ask = "Gumawa ka ng dalawang tula tungkol sa GinPom cocktail, tapos isalin mo sa English.";
var plan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine(plan);

{{!-- Step 1: Identify key values --}}
{{set "ginpomPoem1" "Sa mataas na landas, ika'y lulan Ang GinPom cocktail, timpla'y tinubuan"}}

{{set "ginpomPoem2" "Ligaya sa dalangin, init nang sumilang GinPom sa labi, saya'y nakakalunod"}}

{{!-- Step 2: Translate poems to English --}}
{{set "englishPoem1" (WriterPlugin-Translate input=ginpomPoem1 language="en")}}

{{set "englishPoem2" (WriterPlugin-Translate input=ginpomPoem2 language="en")}}

{{!-- Step 3: Output the translated poems --}}
{{json (concat "Poem 1 (English): " englishPoem1)}}
{{json (concat "Poem 2 (English): " englishPoem2)}}


### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [5]:
#pragma warning disable SKEXP0060

var result = await plan.InvokeAsync(kernel);
result


Poem 1 (English): In the highlands, you are aboard the GinPom cocktail, mixed with natural ingredients.
Poem 2 (English): Joy in prayer, the warmth of the rising sun, GinPom on the lips, happiness overwhelms.